<a href="https://colab.research.google.com/github/maimonahST/GP/blob/main/Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict
from sklearn.base import clone
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, concatenate, Dense, MaxPooling1D, Flatten, GRU, SimpleRNN, LSTM, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential

In [13]:
from sklearn.model_selection import GroupKFold
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import numpy as np

gkf=GroupKFold()
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [15]:
def two_splits(data_array , label_array, group_array):

  # Split data_array into train and validate
  train_groups = []
  for train_index, test_index in gkf.split(data_array, label_array, groups=group_array):
      X_train, y_train = data_array[train_index],label_array[train_index]
      X_test, y_test =data_array[test_index],label_array[test_index]
      train_groups = group_array[train_index]

      scaler=StandardScaler3D()
      X_train=scaler.fit_transform(X_train)
      X_test=scaler.transform(X_test)

      print(f"group={np.unique(group_array[train_index])}")
      print(f"group={np.unique(group_array[test_index])}")
      break

  return X_train, X_test, y_train, y_test, train_groups

In [18]:
def block(input):
    conv1 = Conv1D(32, 2, strides=2, activation='relu', padding="same")(input)
    conv2 = Conv1D(32, 4, strides=2, activation='relu', padding="causal")(input)
    conv3 = Conv1D(32, 8, strides=2, activation='relu', padding="causal")(input)
    x = concatenate([conv1, conv2, conv3], axis=2)
    return x

# ICA

**Prepare the data**


In [ ]:
X_ica  = np.load('/content/drive/MyDrive/GP/GP-Data/X_ica2.npy')
y_ica = np.load('/content/drive/MyDrive/GP/GP-Data/y_ica2.npy')
y_ica = y_ica.astype('int')

X_ica.shape, y_ica.shape

((45, 32, 32), (45,))

In [ ]:
np.unique(y_ica)

array([0, 1, 2])

In [ ]:
# Load your dataset and split into training and testing sets
X_train_ica, X_test_ica, y_train_ica, y_test_ica = train_test_split(X_ica, y_ica, test_size=0.2, random_state=101)


y_train_ica[y_train_ica==0] = 1 #Stroke
y_train_ica[y_train_ica==2] = 0 #Healthy

y_test_ica[y_test_ica==0] = 1 #Stroke
y_test_ica[y_test_ica==2] = 0 #Healthy


np.unique(y_train_ica), np.unique(y_test_ica)


(array([0, 1]), array([0, 1]))

In [ ]:
# Scale the data using StandardScaler
scaler = StandardScaler3D()
X_train_ica = scaler.fit_transform(X_train_ica)
X_test_ica = scaler.transform(X_test_ica)

# Flatten the data array for ML models
X_train_ica = X_train_ica.reshape(X_train_ica.shape[0], -1)
X_test_ica = X_test_ica.reshape(X_test_ica.shape[0], -1)
y_train_ica = y_train_ica.reshape(y_train_ica.shape[0], -1)
y_test_ica = y_test_ica.reshape(y_test_ica.shape[0], -1)


IndexError: tuple index out of range

## Ensemble

Ensemble ML models (Bernoulli NB, Adaboost, XGBoost, DT) using several ensemble methods

In [ ]:
# Train ML models (Base models)
bernoulli_nb = BernoulliNB(alpha=0.1, binarize = 0.0 , fit_prior = True)
bernoulli_nb.fit(X_train_ica, y_train_ica)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BernoulliNB(alpha=0.1)

In [ ]:
Adaboost = AdaBoostClassifier(n_estimators = 200 , learning_rate = 1)
Adaboost.fit(X_train_ica, y_train_ica)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier(learning_rate=1, n_estimators=200)

In [ ]:
xgboost = XGBClassifier(max_depth=9, n_estimators=200, learning_rate=0.01)
xgboost.fit(X_train_ica, y_train_ica)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
decision_tree = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_leaf=5)
decision_tree.fit(X_train_ica, y_train_ica)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=5)

In [ ]:
RandomForest = RandomForestClassifier(n_estimators=100, max_depth=3, min_samples_leaf=5)
RandomForest.fit(X_train_ica, y_train_ica)

<ipython-input-19-1edf83764416>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RandomForest.fit(X_train_ica, y_train_ica)


RandomForestClassifier(max_depth=3, min_samples_leaf=5)

In [ ]:
# 4. Generate predictions
bernoulli_nb_predictions_ica = bernoulli_nb.predict(X_test_ica).reshape(-1, 1)
dt_predictions_ica = decision_tree.predict(X_test_ica).reshape(-1, 1)
xgboost_predictions_ica = xgboost.predict(X_test_ica).reshape(-1, 1)
RandomForest_predictions_ica= RandomForest.predict(X_test_ica).reshape(-1, 1)
Adaboost_predictions_ica = Adaboost.predict(X_test_ica).reshape(-1, 1)

bernoulli_nb_predictions_ica.shape , dt_predictions_ica.shape , xgboost_predictions_ica.shape, RandomForest_predictions_ica.shape

((9, 1), (9, 1), (9, 1), (9, 1))

#### Voting

In [ ]:
# bernoulli_nb_predictions, xgboost_predictions,dt_predictions,RandomForest_predictions
from scipy.stats import mode

threshold = 0.5
bernoulli_nb_predictions_ica = (bernoulli_nb_predictions_ica >= threshold).astype(int)
dt_predictions_ica =  (dt_predictions_ica >= threshold).astype(int)
xgboost_predictions_ica =  (xgboost_predictions_ica >= threshold).astype(int)
Adaboost_predictions_ica =  (Adaboost_predictions_ica >= threshold).astype(int)



ensemble_list_ica = [bernoulli_nb_predictions_ica,
                 xgboost_predictions_ica,
                 dt_predictions_ica,
                 Adaboost_predictions_ica]

# Combine predictions
ensemble_voting_ica = mode(ensemble_list_ica, axis=0)[0].squeeze()


# Compute accuracy using the binary class labels
ensemble_accuracy_ica = accuracy_score(y_test_ica, ensemble_voting_ica)
precision_ica = precision_score(y_test_ica, ensemble_voting_ica)
recall_ica = recall_score(y_test_ica, ensemble_voting_ica)
f1_ica = f1_score(y_test_ica, ensemble_voting_ica)

print(f'Ensemble accuracy : {ensemble_accuracy_ica}')
print(f'Ensemble precision : {precision_ica}')
print(f'Ensemble recall : {recall_ica}')
print(f'Ensemble f1 : {f1_ica}')


Ensemble accuracy : 0.8888888888888888
Ensemble precision : 1.0
Ensemble recall : 0.8333333333333334
Ensemble f1 : 0.9090909090909091


#### Stacking

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb


bernoulli_nb_train = bernoulli_nb.predict(X_train_ica).reshape(-1, 1)
dt_train = decision_tree.predict(X_train_ica).reshape(-1, 1)
xgboost_train = xgboost.predict(X_train_ica).reshape(-1, 1)
RandomForest_train= RandomForest.predict(X_train_ica).reshape(-1, 1)
Adaboost_train = Adaboost.predict(X_train_ica).reshape(-1, 1)


#making a new dataset for training our final model by stacking the predictions on the validation data
train_stack = np.column_stack((bernoulli_nb_train,
                               dt_train,
                               xgboost_train,
                               Adaboost_train))

#making the final test set for our final model by stacking the predictions on the test data
test_stack = np.column_stack((bernoulli_nb_predictions_ica,
                              xgboost_predictions_ica,
                              dt_predictions_ica,
                              Adaboost_predictions_ica))


final_model = RandomForestClassifier()

#training the final model on the stacked predictions
final_model.fit(train_stack,y_train_ica)

final_predictions = final_model.predict(test_stack)

# Compute accuracy using the binary class labels
ensemble_accuracy_ica = accuracy_score(y_test_ica, final_predictions)
precision_ica = precision_score(y_test_ica, final_predictions)
recall_ica = recall_score(y_test_ica, final_predictions)
f1_ica = f1_score(y_test_ica, final_predictions)

print(f'Ensemble accuracy : {ensemble_accuracy_ica}')
print(f'Ensemble precision : {precision_ica}')
print(f'Ensemble recall : {recall_ica}')
print(f'Ensemble f1 : {f1_ica}')

Ensemble accuracy : 0.8888888888888888
Ensemble precision : 1.0
Ensemble recall : 0.8333333333333334
Ensemble f1 : 0.9090909090909091


<ipython-input-23-eb5337c8d226>:29: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  final_model.fit(train_stack,y_train_ica)


# Epochs

In [ ]:
data_array = np.load('/content/drive/MyDrive/GP/GP-Data/data_array.npy')
label_array = np.load('/content/drive/MyDrive/GP/GP-Data/label_array.npy')
group_array = np.load('/content/drive/MyDrive/GP/GP-Data/group_array.npy')

label_array = label_array.astype('int')

data_array.shape, label_array.shape, group_array.shape

((7150, 1250, 33), (7150,), (7150,))

In [ ]:
np.unique(label_array)

array([0, 1, 2])

In [ ]:
# Load your dataset and split into training and testing sets
X_train, X_test, y_train, y_test, _= two_splits(data_array, label_array, group_array)


y_train[y_train==0] = 1 #Stroke
y_train[y_train==2] = 0 #Healthy

y_test[y_test==0] = 1 #Stroke
y_test[y_test==2] = 0 #Healthy


group=[ 0  1  2  3  4  5  6  7  8 10 11 12 13 15 16 18 19 20 22 23 25 27 28 29
 30 31 34 36 37 38 39 40 41 42 43 44 45 46 47]
group=[ 9 14 17 21 24 26 32 33 35]


In [ ]:
np.unique(y_train), np.unique(y_test)

(array([0, 1]), array([0, 1]))

In [ ]:
# Flatten the data array for ML models
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
# y_train = y_train.reshape(y_train.shape[0], -1)
# y_test = y_test.reshape(y_test.shape[0], -1)

In [ ]:
# Train ML models (Base models)
bernoulli_nb = BernoulliNB(alpha=0.1, binarize = 0.0 , fit_prior = True)
bernoulli_nb.fit(X_train, y_train)

BernoulliNB(alpha=0.1)

In [ ]:
Adaboost = AdaBoostClassifier(n_estimators = 200 , learning_rate = 1)
Adaboost.fit(X_train, y_train)

AdaBoostClassifier(learning_rate=1, n_estimators=200)

In [ ]:
xgboost = XGBClassifier(max_depth=9, n_estimators=200, learning_rate=0.01)
xgboost.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
decision_tree = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_leaf=5)
decision_tree.fit(X_train, y_train)

In [ ]:
RandomForest = RandomForestClassifier(n_estimators=100, max_depth=3, min_samples_leaf=5)
RandomForest.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, min_samples_leaf=5)

In [ ]:
# 4. Generate predictions
bernoulli_nb_predictions_train  = bernoulli_nb.predict(X_train).reshape(-1, 1)
dt_predictions_train            = decision_tree.predict(X_train).reshape(-1, 1)
xgboost_predictions_train       = xgboost.predict(X_train).reshape(-1, 1)
RandomForest_predictions_train  = RandomForest.predict(X_train).reshape(-1, 1)
Adaboost_predictions_train      = Adaboost.predict(X_train).reshape(-1, 1)


bernoulli_nb_predictions_test   = bernoulli_nb.predict(X_test).reshape(-1, 1)
dt_predictions_test             = decision_tree.predict(X_test).reshape(-1, 1)
xgboost_predictions_test        = xgboost.predict(X_test).reshape(-1, 1)
RandomForest_predictions_test   = RandomForest.predict(X_test).reshape(-1, 1)
Adaboost_predictions_test       = Adaboost.predict(X_test).reshape(-1, 1)



### Stacking ML only

In [ ]:
#making a new dataset for training our final model by stacking the predictions on the validation data
train_stack = np.column_stack((bernoulli_nb_predictions_train,
                               xgboost_predictions_train,
                               Adaboost_predictions_train))

#making the final test set for our final model by stacking the predictions on the test data
test_stack = np.column_stack((bernoulli_nb_predictions_test,
                              xgboost_predictions_test,
                              Adaboost_predictions_test))


final_model = RandomForestClassifier()

#training the final model on the stacked predictions
final_model.fit(train_stack,y_train)

final_predictions = final_model.predict(test_stack)

# Compute accuracy using the binary class labels
ensemble_accuracy = accuracy_score(y_test, final_predictions)
precision         = precision_score(y_test, final_predictions)
recall            = recall_score(y_test, final_predictions)
f1                = f1_score(y_test, final_predictions)

print(f'Ensemble accuracy : {ensemble_accuracy}')
print(f'Ensemble precision : {precision}')
print(f'Ensemble recall : {recall}')
print(f'Ensemble f1 : {f1}')

Ensemble accuracy : 0.6854663774403471
Ensemble precision : 0.6881878209831255
Ensemble recall : 0.989451476793249
Ensemble f1 : 0.8117697966248377


## Ensemble DL and ML

In [ ]:
# 1. Preprocess the data for both ML and DL models

# Load your dataset and split into training and testing sets
X_train, X_test, y_train, y_test, train_groups = two_splits(data_array, label_array, group_array)


y_train[y_train==0] = 1 #Stroke
y_train[y_train==2] = 0 #Healthy

y_test[y_test==0] = 1 #Stroke
y_test[y_test==2] = 0 #Healthy

group=[ 0  1  2  3  4  5  6  7  8 10 11 12 13 15 16 18 19 20 22 23 25 27 28 29
 30 31 34 36 37 38 39 40 41 42 43 44 45 46 47]
group=[ 9 14 17 21 24 26 32 33 35]


In [ ]:
X_train_ml, y_train_ml = X_train.reshape(X_train.shape[0], -1), y_train

In [ ]:
# 2. Train ML models

# bernoulli_nb = BernoulliNB(alpha=0.1, binarize = 0.0 , fit_prior = True))
# bernoulli_nb.fit(X_train_ml, y_train_ml)

decision_tree = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_leaf=5)
decision_tree.fit(X_train_ml, y_train_ml)



DecisionTreeClassifier(max_depth=3, min_samples_leaf=5)

In [ ]:
xgboost = XGBClassifier(max_depth=3, n_estimators=200, learning_rate=0.1, random_state=42)
xgboost.fit(X_train_ml, y_train_ml)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
# DL
X_train_dl, X_val_dl, y_train_dl, y_val_dl, _ = two_splits(X_train, y_train, train_groups)

group=[ 0  2  3  5  6  7 10 11 12 13 15 16 18 20 22 23 25 27 28 30 31 34 36 38
 39 40 41 42 43 44 46 47]
group=[ 1  4  8 19 29 37 45]


In [ ]:
X_train_dl.shape, y_train_dl.shape, X_val_dl.shape, y_val_dl.shape

((4728, 1250, 33), (4728,), (1039, 1250, 33), (1039,))

In [ ]:
# 3. Train DL models

# Define the input layer with a shape of (1250, 33)
input_layer = Input(shape=(1250, 33))

# Create the first block
block1 = block(input_layer)

# Create the second block using the 'block' function with the output of the first block as input
block2 = block(block1)

# Create the third block using the 'block' function with the output of the second block as input
block3 = block(block2)

cnn_out1 = Conv1D(32, 3, activation='relu', padding='same')(block3)
cnn_out2 = Conv1D(32, 3, activation='relu', padding='same')(cnn_out1)
cnn_out3 = MaxPooling1D(pool_size=2)(cnn_out2)
cnn_out4 = Flatten()(cnn_out3)

# Create a Dense layer with 1 unit and sigmoid activation for binary classification
predictions = Dense(1, activation='sigmoid')(cnn_out4)

# Define the model with 'input_layer' as the input and 'predictions' as the output
CNN = Model(inputs=input_layer, outputs=predictions)

# Specify the Adam optimizer with a learning rate of 0.0001
optimizer = Adam(learning_rate=0.0001)

# Compile the model with binary crossentropy loss and accuracy as the evaluation metric
CNN.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model using training features and labels
hist = CNN.fit(X_train_dl, y_train_dl, epochs=70, batch_size=132, validation_data= (X_val_dl, y_val_dl), callbacks=[early_stopping])


Epoch 1/70
36/36 [==============================] - 13s 130ms/step - loss: 0.6888 - accuracy: 0.5791 - val_loss: 0.6903 - val_accuracy: 0.5544
Epoch 2/70
36/36 [==============================] - 1s 38ms/step - loss: 0.6650 - accuracy: 0.6036 - val_loss: 0.6893 - val_accuracy: 0.5515
Epoch 3/70
36/36 [==============================] - 1s 38ms/step - loss: 0.6498 - accuracy: 0.6191 - val_loss: 0.6895 - val_accuracy: 0.5563
Epoch 4/70
36/36 [==============================] - 1s 38ms/step - loss: 0.6346 - accuracy: 0.6436 - val_loss: 0.6896 - val_accuracy: 0.5573
Epoch 5/70
36/36 [==============================] - 1s 38ms/step - loss: 0.6186 - accuracy: 0.6548 - val_loss: 0.6878 - val_accuracy: 0.5650
Epoch 6/70
36/36 [==============================] - 1s 38ms/step - loss: 0.5967 - accuracy: 0.6887 - val_loss: 0.6864 - val_accuracy: 0.5659
Epoch 7/70
36/36 [==============================] - 1s 38ms/step - loss: 0.5704 - accuracy: 0.7250 - val_loss: 0.6898 - val_accuracy: 0.5794
Epoch 8/70


In [ ]:
# Define the input layer with a shape of (1250, 33)
input = Input(shape=(1250, 33))

# Create the first block
block1 = block(input)

# Create the second block using the 'block' function with the output of the first block as input
block2 = block(block1)

# Create the third block using the 'block' function with the output of the second block as input
block3 = block(block2)


gru_out1 = GRU(32,activation='tanh',return_sequences=True)(block3)
gru_out2 = GRU(32,activation='tanh',return_sequences=True)(gru_out1)
gru_out = concatenate([gru_out1,gru_out2],axis=2)
gru_out3 = GRU(32,activation='tanh',return_sequences=True)(gru_out)
gru_out = concatenate([gru_out1,gru_out2,gru_out3])
gru_out4 = GRU(32,activation='tanh')(gru_out)


# Create a Dense layer with 1 unit and sigmoid activation for binary classification
predictions = Dense(1, activation='sigmoid')(gru_out4)

# Define the model with 'input' as the input and 'predictions' as the output
CNN_GRU = Model(inputs=input, outputs=predictions)

# Specify the Adam optimizer with a learning rate of 0.0001
optimizer = Adam(learning_rate=0.0001)

# Compile the model with binary crossentropy loss and accuracy as the evaluation metric
CNN_GRU.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model using training features and labels
hist = CNN_GRU.fit(X_train_dl, y_train_dl, epochs=70, batch_size=132,
                   validation_data= (X_val_dl, y_val_dl), callbacks=early_stopping)

Epoch 1/70
36/36 [==============================] - 11s 106ms/step - loss: 0.6806 - accuracy: 0.5825 - val_loss: 0.6845 - val_accuracy: 0.5775
Epoch 2/70
36/36 [==============================] - 2s 58ms/step - loss: 0.6690 - accuracy: 0.5941 - val_loss: 0.6840 - val_accuracy: 0.5765
Epoch 3/70
36/36 [==============================] - 2s 56ms/step - loss: 0.6597 - accuracy: 0.5990 - val_loss: 0.6827 - val_accuracy: 0.5804
Epoch 4/70
36/36 [==============================] - 2s 58ms/step - loss: 0.6424 - accuracy: 0.6178 - val_loss: 0.6804 - val_accuracy: 0.5611
Epoch 5/70
36/36 [==============================] - 2s 57ms/step - loss: 0.6145 - accuracy: 0.6527 - val_loss: 0.6793 - val_accuracy: 0.5650
Epoch 6/70
36/36 [==============================] - 2s 56ms/step - loss: 0.5685 - accuracy: 0.6992 - val_loss: 0.6833 - val_accuracy: 0.5784
Epoch 7/70
36/36 [==============================] - 2s 57ms/step - loss: 0.5095 - accuracy: 0.7434 - val_loss: 0.6846 - val_accuracy: 0.6006
Epoch 8/70


In [ ]:
RNN = Sequential()
RNN.add(SimpleRNN(128, input_shape=(1250, 33), return_sequences=True))  # RNN layer with 128 units and return sequences
RNN.add(SimpleRNN(128))  # Additional RNN layer with 128 units
RNN.add(Dense(64, activation='relu'))  # Dense layer with 64 units and ReLU activation
RNN.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation

# Compile the model

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
RNN.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history=RNN.fit(X_train_dl, y_train_dl, epochs=100, batch_size=132,validation_data= (X_val_dl, y_val_dl), callbacks=[early_stopping])


Epoch 1/100
36/36 [==============================] - 56s 2s/step - loss: 0.6816 - accuracy: 0.5802 - val_loss: 0.7153 - val_accuracy: 0.5197
Epoch 2/100
36/36 [==============================] - 50s 1s/step - loss: 0.6403 - accuracy: 0.6320 - val_loss: 0.7171 - val_accuracy: 0.5140
Epoch 3/100
36/36 [==============================] - 51s 1s/step - loss: 0.6097 - accuracy: 0.6717 - val_loss: 0.7203 - val_accuracy: 0.5120
Epoch 4/100
36/36 [==============================] - 50s 1s/step - loss: 0.5784 - accuracy: 0.7064 - val_loss: 0.7249 - val_accuracy: 0.5034


In [ ]:
# 4. Generate predictions
X_test_ml, X_test_dl = X_test.reshape(X_test.shape[0], -1), X_test

xgboost_predictions_train = xgboost.predict(X_train_ml).reshape(-1, 1)
dt_predictions_train = decision_tree.predict(X_train_ml).reshape(-1, 1)
CNN_predictions_train = CNN.predict(X_train).reshape(-1, 1)
CNN_GRU_predictions_train = CNN_GRU.predict(X_train).reshape(-1, 1)
RNN_predictions_train = RNN.predict(X_train).reshape(-1, 1)


xgboost_predictions_test = xgboost.predict(X_test_ml).reshape(-1, 1)
dt_predictions_test = decision_tree.predict(X_test_ml).reshape(-1, 1)
CNN_predictions_test = CNN.predict(X_test_dl).reshape(-1, 1)
CNN_GRU_predictions_test = CNN_GRU.predict(X_test_dl).reshape(-1, 1)
RNN_predictions_test = RNN.predict(X_test_dl).reshape(-1, 1)


44/44 [==============================] - 7s 162ms/step


In [ ]:
# xgboost
ensemble_accuracy = accuracy_score(y_test, xgboost_predictions_test)
precision         = precision_score(y_test, xgboost_predictions_test)
recall            = recall_score(y_test, xgboost_predictions_test)
f1                = f1_score(y_test, xgboost_predictions_test)

print(f'xgboost accuracy : {ensemble_accuracy}')
print(f'xgboost precision : {precision}')
print(f'xgboost recall : {recall}')
print(f'xgboost f1 : {f1}')


# dt
ensemble_accuracy = accuracy_score(y_test, dt_predictions_test)
precision         = precision_score(y_test, dt_predictions_test)
recall            = recall_score(y_test, dt_predictions_test)
f1                = f1_score(y_test, dt_predictions_test)

print(f'dt accuracy : {ensemble_accuracy}')
print(f'dt precision : {precision}')
print(f'dt recall : {recall}')
print(f'dt f1 : {f1}')


# CNN
threshhold = 0.5
CNN_predictions_test = np.where(CNN_predictions_test > threshhold, 1, 0)
ensemble_accuracy = accuracy_score(y_test, CNN_predictions_test)
precision         = precision_score(y_test, CNN_predictions_test)
recall            = recall_score(y_test, CNN_predictions_test)
f1                = f1_score(y_test, CNN_predictions_test)

print(f'CNN accuracy : {ensemble_accuracy}')
print(f'CNN precision : {precision}')
print(f'CNN recall : {recall}')
print(f'CNN f1 : {f1}')


# CNN-GRU
threshhold = 0.5
CNN_GRU_predictions_test = np.where(CNN_GRU_predictions_test > threshhold, 1, 0)
ensemble_accuracy = accuracy_score(y_test, CNN_predictions_test)
precision         = precision_score(y_test, CNN_predictions_test)
recall            = recall_score(y_test, CNN_predictions_test)
f1                = f1_score(y_test, CNN_predictions_test)

print(f'CNN-GRU accuracy : {ensemble_accuracy}')
print(f'CNN-GRU precision : {precision}')
print(f'CNN-GRU recall : {recall}')
print(f'CNN-GRU f1 : {f1}')

# RNN
threshhold = 0.5
RNN_predictions_test = np.where(RNN_predictions_test > threshhold, 1, 0)
ensemble_accuracy = accuracy_score(y_test, RNN_predictions_test)
precision         = precision_score(y_test, RNN_predictions_test)
recall            = recall_score(y_test, RNN_predictions_test)
f1                = f1_score(y_test, RNN_predictions_test)

print(f'RNN accuracy : {ensemble_accuracy}')
print(f'RNN precision : {precision}')
print(f'RNN recall : {recall}')
print(f'RNN f1 : {f1}')


xgboost accuracy : 0.7845263919016631
xgboost precision : 0.8472222222222222
xgboost recall : 0.8364978902953587
xgboost f1 : 0.8418259023354565
dt accuracy : 0.6789587852494577
dt precision : 0.8103448275862069
dt recall : 0.6940928270042194
dt f1 : 0.7477272727272727
CNN accuracy : 0.6681127982646421
CNN precision : 0.7643243243243243
CNN recall : 0.7457805907172996
CNN f1 : 0.7549386011745862
CNN-GRU accuracy : 0.6681127982646421
CNN-GRU precision : 0.7643243243243243
CNN-GRU recall : 0.7457805907172996
CNN-GRU f1 : 0.7549386011745862
RNN accuracy : 0.6362979031091829
RNN precision : 0.6943231441048034
RNN recall : 0.8386075949367089
RNN f1 : 0.7596751075011945


In [ ]:
#making a new dataset for training our final model by stacking the predictions on the validation data
train_stack = np.column_stack((xgboost_predictions_train,
                               dt_predictions_train,
                               CNN_predictions_train,
                               CNN_GRU_predictions_train,
                               RNN_predictions_train))

#making the final test set for our final model by stacking the predictions on the test data
test_stack = np.column_stack((xgboost_predictions_test,
                              dt_predictions_test,
                              CNN_predictions_test,
                              CNN_GRU_predictions_test,
                              RNN_predictions_test))


final_model = RandomForestClassifier(n_estimators=200)

#training the final model on the stacked predictions
final_model.fit(train_stack,y_train)

final_predictions = final_model.predict(test_stack)

# Compute accuracy using the binary class labels
ensemble_accuracy = accuracy_score(y_test, final_predictions)
precision         = precision_score(y_test, final_predictions)
recall            = recall_score(y_test, final_predictions)
f1                = f1_score(y_test, final_predictions)

print(f'Ensemble accuracy : {ensemble_accuracy}')
print(f'Ensemble precision : {precision}')
print(f'Ensemble recall : {recall}')
print(f'Ensemble f1 : {f1}')

Ensemble accuracy : 0.7845263919016631
Ensemble precision : 0.8472222222222222
Ensemble recall : 0.8364978902953587
Ensemble f1 : 0.8418259023354565


# DT Data

In [10]:
data_array = np.load('/content/drive/MyDrive/GP/GP-Data/data_array.npy')
label_array = np.load('/content/drive/MyDrive/GP/GP-Data/label_array.npy')
group_array = np.load('/content/drive/MyDrive/GP/GP-Data/group_array.npy')

label_array = label_array.astype('int')

data_array.shape, label_array.shape, group_array.shape

((7150, 1250, 33), (7150,), (7150,))

In [16]:
# Load your dataset and split into training and testing sets
X_train, X_test, y_train, y_test, train_groups = two_splits(data_array, label_array, group_array)


y_train[y_train==0] = 1 #Stroke
y_train[y_train==2] = 0 #Healthy

y_test[y_test==0] = 1 #Stroke
y_test[y_test==2] = 0 #Healthy

group=[ 0  1  2  3  4  5  6  7  8 10 11 12 13 15 16 18 19 20 22 23 25 27 28 29
 30 31 34 36 37 38 39 40 41 42 43 44 45 46 47]
group=[ 9 14 17 21 24 26 32 33 35]


In [19]:
X_train_ml = X_train.reshape(X_train.shape[0], -1)
X_test_ml = X_test.reshape(X_test.shape[0], -1)

In [20]:
## featuers reduction using DT
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def select_features_with_decision_tree(X_train, y_train, X_test, y_test, threshold=0.1):
    tree_params = {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 5}
    # Train a Decision Tree Classifier
    dt_classifier = DecisionTreeClassifier(**tree_params)

    # Train a Decision Tree classifier to select important features
    #dt = DecisionTreeClassifier(random_state=42)
    dt_classifier.fit(X_train, y_train)

    # Select features based on importance using the trained Decision Tree
    sfm = SelectFromModel(dt_classifier, threshold=0.1)
    sfm.fit(X_train, y_train)

    # Transform the training and testing data to include only the selected features
    X_train_selected = sfm.transform(X_train)
    X_test_selected = sfm.transform(X_test)

    return X_train_selected, X_test_selected




X_train_selected , X_test_selected = select_features_with_decision_tree(X_train_ml,
                                                                        y_train,
                                                                        X_test_ml,
                                                                        y_test)


In [21]:
X_train_selected.shape , X_test_selected.shape

((5767, 4), (1383, 4))

In [29]:
# 2. Train ML models
decision_tree = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_leaf=5)
decision_tree.fit(X_train_selected, y_train)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=5)

In [30]:
xgboost = XGBClassifier(max_depth=3, n_estimators=200, learning_rate=0.1, random_state=42)
xgboost.fit(X_train_selected, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [31]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(X_train_selected, y_train)

KNeighborsClassifier()

In [27]:
lgbm_classifier = LGBMClassifier(max_depth= -1, n_estimators=200, learning_rate=0.01, subsample = 0.8, colsample_bytree = 0.8 )
lgbm_classifier.fit(X_train_selected, y_train)


[LightGBM] [Info] Number of positive: 3406, number of negative: 2361
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000325 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 230
[LightGBM] [Info] Number of data points in the train set: 5767, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.590602 -> initscore=0.366453
[LightGBM] [Info] Start training from score 0.366453
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(colsample_bytree=0.8, learning_rate=0.01, n_estimators=200,
               subsample=0.8)

In [32]:
# 4. Generate predictions
X_test_ml, X_test_dl = X_test.reshape(X_test.shape[0], -1), X_test

xgboost_predictions_train = xgboost.predict(X_train_selected).reshape(-1, 1)
dt_predictions_train = decision_tree.predict(X_train_selected).reshape(-1, 1)
knn_predictions_train = knn.predict(X_train_selected).reshape(-1, 1)
lgbm_predictions_train = lgbm_classifier.predict(X_train_selected).reshape(-1, 1)


xgboost_predictions_test = xgboost.predict(X_test_selected).reshape(-1, 1)
dt_predictions_test = decision_tree.predict(X_test_selected).reshape(-1, 1)
knn_predictions_test = knn.predict(X_test_selected).reshape(-1, 1)
lgbm_predictions_test = lgbm_classifier.predict(X_test_selected).reshape(-1, 1)



In [33]:
# xgboost
ensemble_accuracy = accuracy_score(y_test, xgboost_predictions_test)
precision         = precision_score(y_test, xgboost_predictions_test)
recall            = recall_score(y_test, xgboost_predictions_test)
f1                = f1_score(y_test, xgboost_predictions_test)

print(f'xgboost accuracy : {ensemble_accuracy}')
print(f'xgboost precision : {precision}')
print(f'xgboost recall : {recall}')
print(f'xgboost f1 : {f1}')


# dt
ensemble_accuracy = accuracy_score(y_test, dt_predictions_test)
precision         = precision_score(y_test, dt_predictions_test)
recall            = recall_score(y_test, dt_predictions_test)
f1                = f1_score(y_test, dt_predictions_test)

print(f'dt accuracy : {ensemble_accuracy}')
print(f'dt precision : {precision}')
print(f'dt recall : {recall}')
print(f'dt f1 : {f1}')


# KNN
ensemble_accuracy = accuracy_score(y_test, knn_predictions_test)
precision         = precision_score(y_test, knn_predictions_test)
recall            = recall_score(y_test, knn_predictions_test)
f1                = f1_score(y_test, knn_predictions_test)


print(f'KNN accuracy : {ensemble_accuracy}')
print(f'KNN precision : {precision}')
print(f'KNN recall : {recall}')
print(f'KNN f1 : {f1}')


# lgbm
ensemble_accuracy = accuracy_score(y_test, lgbm_predictions_test)
precision         = precision_score(y_test, lgbm_predictions_test)
recall            = recall_score(y_test, lgbm_predictions_test)
f1                = f1_score(y_test, lgbm_predictions_test)

print(f'lgbm accuracy : {ensemble_accuracy}')
print(f'lgbm precision : {precision}')
print(f'lgbm recall : {recall}')
print(f'lgbm f1 : {f1}')



xgboost accuracy : 0.6789587852494577
xgboost precision : 0.8134328358208955
xgboost recall : 0.689873417721519
xgboost f1 : 0.7465753424657535
dt accuracy : 0.6789587852494577
dt precision : 0.8103448275862069
dt recall : 0.6940928270042194
dt f1 : 0.7477272727272727
KNN accuracy : 0.779464931308749
KNN precision : 0.8445873526259379
KNN recall : 0.8312236286919831
KNN f1 : 0.837852206273259
lgbm accuracy : 0.6767895878524945
lgbm precision : 0.8096415327564895
lgbm recall : 0.6909282700421941
lgbm f1 : 0.7455890722822995


In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb




#making a new dataset for training our final model by stacking the predictions on the validation data
train_stack = np.column_stack((xgboost_predictions_train,
                               dt_predictions_train,
                               knn_predictions_train,
                               lgbm_predictions_train))

#making the final test set for our final model by stacking the predictions on the test data
test_stack = np.column_stack((xgboost_predictions_test,
                               dt_predictions_test,
                               knn_predictions_test,
                               lgbm_predictions_test))


final_model = RandomForestClassifier(n_estimators= 200)

#training the final model on the stacked predictions
final_model.fit(train_stack, y_train)

final_predictions = final_model.predict(test_stack)

# Compute accuracy using the binary class labels
ensemble_accuracy_dt = accuracy_score(y_test, final_predictions)
precision_dt = precision_score(y_test, final_predictions)
recall_dt = recall_score(y_test, final_predictions)
f1_dt = f1_score(y_test, final_predictions)

print(f'Ensemble accuracy : {ensemble_accuracy_dt}')
print(f'Ensemble precision : {precision_dt}')
print(f'Ensemble recall : {recall_dt}')
print(f'Ensemble f1 : {f1_dt}')

Ensemble accuracy : 0.6789587852494577
Ensemble precision : 0.8134328358208955
Ensemble recall : 0.689873417721519
Ensemble f1 : 0.7465753424657535


# Mixed data

In [35]:
# DL
X_train_dl, X_val_dl, y_train_dl, y_val_dl, _ = two_splits(X_train, y_train, train_groups)

group=[ 0  2  3  5  6  7 10 11 12 13 15 16 18 20 22 23 25 27 28 30 31 34 36 38
 39 40 41 42 43 44 46 47]
group=[ 1  4  8 19 29 37 45]


In [36]:
X_train_dl.shape, y_train_dl.shape, X_val_dl.shape, y_val_dl.shape

((4728, 1250, 33), (4728,), (1039, 1250, 33), (1039,))

In [37]:
def block(input):
    conv1 = Conv1D(32, 2, strides=2, activation='relu', padding="same")(input)
    conv2 = Conv1D(32, 4, strides=2, activation='relu', padding="causal")(input)
    conv3 = Conv1D(32, 8, strides=2, activation='relu', padding="causal")(input)
    x = concatenate([conv1, conv2, conv3], axis=2)
    return x

In [40]:
# 3. Train DL models

# Define the input layer with a shape of (1250, 33)
input_layer = Input(shape=(1250,33))

# Create the first block
block1 = block(input_layer)

# Create the second block using the 'block' function with the output of the first block as input
block2 = block(block1)

# Create the third block using the 'block' function with the output of the second block as input
block3 = block(block2)

cnn_out1 = Conv1D(32, 3, activation='tanh', padding='same')(block3)
cnn_out2 = Conv1D(32, 3, activation='tanh', padding='same')(cnn_out1)
cnn_out3 = MaxPooling1D(pool_size=2)(cnn_out2)
cnn_out4 = Flatten()(cnn_out3)

# Create a Dense layer with 1 unit and sigmoid activation for binary classification
predictions = Dense(1, activation='sigmoid')(cnn_out4)

# Define the model with 'input_layer' as the input and 'predictions' as the output
CNN = Model(inputs=input_layer, outputs=predictions)

# Specify the Adam optimizer with a learning rate of 0.0001
optimizer = Adam(learning_rate=0.0001)

# Compile the model with binary crossentropy loss and accuracy as the evaluation metric
CNN.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model using training features and labels
hist = CNN.fit(X_train_dl, y_train_dl, epochs=20, batch_size=132, validation_data= (X_val_dl, y_val_dl), callbacks=[early_stopping])


Epoch 1/20
36/36 [==============================] - 14s 137ms/step - loss: 0.6947 - accuracy: 0.5664 - val_loss: 0.6985 - val_accuracy: 0.5525
Epoch 2/20
36/36 [==============================] - 1s 39ms/step - loss: 0.6667 - accuracy: 0.6003 - val_loss: 0.7003 - val_accuracy: 0.5602
Epoch 3/20
36/36 [==============================] - 1s 40ms/step - loss: 0.6474 - accuracy: 0.6231 - val_loss: 0.6977 - val_accuracy: 0.5544
Epoch 4/20
36/36 [==============================] - 1s 39ms/step - loss: 0.6264 - accuracy: 0.6508 - val_loss: 0.6987 - val_accuracy: 0.5303
Epoch 5/20
36/36 [==============================] - 1s 40ms/step - loss: 0.6026 - accuracy: 0.6838 - val_loss: 0.6948 - val_accuracy: 0.5428
Epoch 6/20
36/36 [==============================] - 1s 40ms/step - loss: 0.5750 - accuracy: 0.7073 - val_loss: 0.6890 - val_accuracy: 0.5582
Epoch 7/20
36/36 [==============================] - 1s 40ms/step - loss: 0.5300 - accuracy: 0.7843 - val_loss: 0.6898 - val_accuracy: 0.5852
Epoch 8/20


In [42]:
# Define the input layer with a shape of (1250, 33)
input = Input(shape=(1250, 33))

# Create the first block
block1 = block(input)

# Create the second block using the 'block' function with the output of the first block as input
block2 = block(block1)

# Create the third block using the 'block' function with the output of the second block as input
block3 = block(block2)


gru_out1 = GRU(32,activation='tanh',return_sequences=True)(block3)
gru_out2 = GRU(32,activation='tanh',return_sequences=True)(gru_out1)
gru_out = concatenate([gru_out1,gru_out2],axis=2)
gru_out3 = GRU(32,activation='tanh',return_sequences=True)(gru_out)
gru_out = concatenate([gru_out1,gru_out2,gru_out3])
gru_out4 = GRU(32,activation='tanh')(gru_out)


# Create a Dense layer with 1 unit and sigmoid activation for binary classification
predictions = Dense(1, activation='sigmoid')(gru_out4)

# Define the model with 'input' as the input and 'predictions' as the output
CNN_GRU = Model(inputs=input, outputs=predictions)

# Specify the Adam optimizer with a learning rate of 0.0001
optimizer = Adam(learning_rate=0.001)

# Compile the model with binary crossentropy loss and accuracy as the evaluation metric
CNN_GRU.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model using training features and labels
hist = CNN_GRU.fit(X_train_dl, y_train_dl, epochs=100, batch_size=128,
                   validation_data= (X_val_dl, y_val_dl), callbacks=early_stopping)


Epoch 1/100
36/36 [==============================] - 12s 104ms/step - loss: 0.6747 - accuracy: 0.5895 - val_loss: 0.6770 - val_accuracy: 0.5756
Epoch 2/100
36/36 [==============================] - 2s 57ms/step - loss: 0.5565 - accuracy: 0.7035 - val_loss: 0.6582 - val_accuracy: 0.6121
Epoch 3/100
36/36 [==============================] - 2s 57ms/step - loss: 0.3793 - accuracy: 0.8238 - val_loss: 0.5921 - val_accuracy: 0.6795
Epoch 4/100
36/36 [==============================] - 2s 57ms/step - loss: 0.2392 - accuracy: 0.9031 - val_loss: 0.5106 - val_accuracy: 0.7584
Epoch 5/100
36/36 [==============================] - 2s 57ms/step - loss: 0.0968 - accuracy: 0.9676 - val_loss: 0.6600 - val_accuracy: 0.7709
Epoch 6/100
36/36 [==============================] - 2s 57ms/step - loss: 0.1806 - accuracy: 0.9321 - val_loss: 0.5959 - val_accuracy: 0.7719
Epoch 7/100
36/36 [==============================] - 2s 57ms/step - loss: 0.0295 - accuracy: 0.9926 - val_loss: 0.6731 - val_accuracy: 0.7729
Epoc

In [53]:
# 4. Generate predictions
X_test_ml, X_test_dl = X_test.reshape(X_test.shape[0], -1), X_test


knn_predictions_train = knn.predict(X_train_selected).reshape(-1, 1)
CNN_predictions_train = CNN.predict(X_train).reshape(-1, 1)
CNN_GRU_predictions_train = CNN_GRU.predict(X_train).reshape(-1, 1)

knn_predictions_test = knn.predict(X_test_selected).reshape(-1, 1)
CNN_predictions_test = CNN.predict(X_test_dl).reshape(-1, 1)
CNN_GRU_predictions_test = CNN_GRU.predict(X_test_dl).reshape(-1, 1)


knn_predictions_test.shape, CNN_predictions_test.shape, CNN_GRU_predictions_test.shape

44/44 [==============================] - 1s 12ms/step


((1383, 1), (1383, 1), (1383, 1))

In [54]:
# KNN
ensemble_accuracy = accuracy_score(y_test, knn_predictions_test)
precision         = precision_score(y_test, knn_predictions_test)
recall            = recall_score(y_test, knn_predictions_test)
f1                = f1_score(y_test, knn_predictions_test)


print(f'KNN accuracy : {ensemble_accuracy}')
print(f'KNN precision : {precision}')
print(f'KNN recall : {recall}')
print(f'KNN f1 : {f1}')



# CNN
threshhold = 0.5
CNN_predictions_test = np.where(CNN_predictions_test > threshhold, 1, 0)
ensemble_accuracy = accuracy_score(y_test, CNN_predictions_test)
precision         = precision_score(y_test, CNN_predictions_test)
recall            = recall_score(y_test, CNN_predictions_test)
f1                = f1_score(y_test, CNN_predictions_test)

print(f'CNN accuracy : {ensemble_accuracy}')
print(f'CNN precision : {precision}')
print(f'CNN recall : {recall}')
print(f'CNN f1 : {f1}')


# CNN-GRU
threshhold = 0.5
CNN_GRU_predictions_test = np.where(CNN_GRU_predictions_test > threshhold, 1, 0)
ensemble_accuracy = accuracy_score(y_test, CNN_predictions_test)
precision         = precision_score(y_test, CNN_predictions_test)
recall            = recall_score(y_test, CNN_predictions_test)
f1                = f1_score(y_test, CNN_predictions_test)

print(f'CNN-GRU accuracy : {ensemble_accuracy}')
print(f'CNN-GRU precision : {precision}')
print(f'CNN-GRU recall : {recall}')
print(f'CNN-GRU f1 : {f1}')



KNN accuracy : 0.779464931308749
KNN precision : 0.8445873526259379
KNN recall : 0.8312236286919831
KNN f1 : 0.837852206273259
CNN accuracy : 0.6334056399132321
CNN precision : 0.7373519913885899
CNN recall : 0.7225738396624473
CNN f1 : 0.7298881193393714
CNN-GRU accuracy : 0.6334056399132321
CNN-GRU precision : 0.7373519913885899
CNN-GRU recall : 0.7225738396624473
CNN-GRU f1 : 0.7298881193393714


In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb



#making a new dataset for training our final model by stacking the predictions on the validation data
train_stack = np.column_stack((knn_predictions_train,
                               CNN_predictions_train,
                               CNN_GRU_predictions_train))

#making the final test set for our final model by stacking the predictions on the test data
test_stack = np.column_stack((knn_predictions_test,
                               CNN_predictions_test,
                               CNN_GRU_predictions_test))


final_model = RandomForestClassifier(n_estimators= 200)

#training the final model on the stacked predictions
final_model.fit(train_stack, y_train)

final_predictions = final_model.predict(test_stack)

# Compute accuracy using the binary class labels
ensemble_accuracy_dt = accuracy_score(y_test, final_predictions)
precision_dt = precision_score(y_test, final_predictions)
recall_dt = recall_score(y_test, final_predictions)
f1_dt = f1_score(y_test, final_predictions)

print(f'Ensemble accuracy : {ensemble_accuracy_dt}')
print(f'Ensemble precision : {precision_dt}')
print(f'Ensemble recall : {recall_dt}')
print(f'Ensemble f1 : {f1_dt}')

Ensemble accuracy : 0.7093275488069414
Ensemble precision : 0.7820247933884298
Ensemble recall : 0.7985232067510548
Ensemble f1 : 0.790187891440501
